## 0. Базовий опис класів Trie та TrieNode із лекції

In [1]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.value = None


class Trie:
    def __init__(self):
        self.root = TrieNode()
        self.size = 0

    def put(self, key, value=None):
        if not isinstance(key, str) or not key:
            raise TypeError(f"Illegal argument for put: key = {key} must be a non-empty string")

        current = self.root
        for char in key:
            if char not in current.children:
                current.children[char] = TrieNode()
            current = current.children[char]
        if current.value is None:
            self.size += 1
        current.value = value

    def get(self, key):
        if not isinstance(key, str) or not key:
            raise TypeError(f"Illegal argument for get: key = {key} must be a non-empty string")

        current = self.root
        for char in key:
            if char not in current.children:
                return None
            current = current.children[char]
        return current.value

    def delete(self, key):
        if not isinstance(key, str) or not key:
            raise TypeError(f"Illegal argument for delete: key = {key} must be a non-empty string")

        def _delete(node, key, depth):
            if depth == len(key):
                if node.value is not None:
                    node.value = None
                    self.size -= 1
                    return len(node.children) == 0
                return False

            char = key[depth]
            if char in node.children:
                should_delete = _delete(node.children[char], key, depth + 1)
                if should_delete:
                    del node.children[char]
                    return len(node.children) == 0 and node.value is None
            return False

        return _delete(self.root, key, 0)

    def is_empty(self):
        return self.size == 0

    def longest_prefix_of(self, s):
        if not isinstance(s, str) or not s:
            raise TypeError(f"Illegal argument for longestPrefixOf: s = {s} must be a non-empty string")

        current = self.root
        longest_prefix = ""
        current_prefix = ""
        for char in s:
            if char in current.children:
                current = current.children[char]
                current_prefix += char
                if current.value is not None:
                    longest_prefix = current_prefix
            else:
                break
        return longest_prefix

    def keys_with_prefix(self, prefix):
        if not isinstance(prefix, str):
            raise TypeError(f"Illegal argument for keysWithPrefix: prefix = {prefix} must be a string")

        current = self.root
        for char in prefix:
            if char not in current.children:
                return []
            current = current.children[char]

        result = []
        self._collect(current, list(prefix), result)
        return result

    def _collect(self, node, path, result):
        if node.value is not None:
            result.append("".join(path))
        for char, next_node in node.children.items():
            path.append(char)
            self._collect(next_node, path, result)
            path.pop()

    def keys(self):
        result = []
        self._collect(self.root, [], result)
        return result


---
## Задача 1. Розширення функціоналу префіксного дерева

#### Реалізуйте два додаткових методи для класу Trie:

- `count_words_with_suffix(pattern)` для підрахунку кількості слів, що закінчуються заданим шаблоном;
- `has_prefix(prefix)` для перевірки наявності слів із заданим префіксом.


#### Технічні умови

- Клас Homework має успадковувати базовий клас `Trie`.
- Методи повинні опрацьовувати помилки введення некоректних даних.
- Вхідні параметри обох методів мають бути рядками.
- Метод `count_words_with_suffix` має повертати ціле число.
- Метод `has_prefix` має повертати булеве значення.

In [2]:
class Homework(Trie):
    def count_words_with_suffix(self, pattern) -> int:
        if not isinstance(pattern, str):
            raise TypeError("The pattern must be a string")
        return sum(word.endswith(pattern) for word in self.keys())

    def has_prefix(self, prefix) -> bool:
        if not isinstance(prefix, str):
            raise TypeError("The prefix must be a string")
        return len(self.keys_with_prefix(prefix)) > 0

### Протестуємо, що вийшло

In [9]:
trie = Homework()
words = ["apple", "application", "banana", "cat"]
for i, word in enumerate(words):
    trie.put(word, i)

assert trie.count_words_with_suffix("e") == 1
assert trie.count_words_with_suffix("ion") == 1
assert trie.count_words_with_suffix("a") == 1
assert trie.count_words_with_suffix("at") == 1

assert trie.has_prefix("app") is True
assert trie.has_prefix("bat") is False
assert trie.has_prefix("ban") is True
assert trie.has_prefix("ca") is True

Код не видав помилок, тобто тести пройдено, але про всяк випадок виведемо результати

In [12]:
print("Task 1 — count_words_with_suffix:")
print("Suffix 'e':", trie.count_words_with_suffix("e"))           # 1
print("Suffix 'ion':", trie.count_words_with_suffix("ion"))       # 1
print("Suffix 'a':", trie.count_words_with_suffix("a"))           # 1
print("Suffix 'at':", trie.count_words_with_suffix("at"))         # 1

print("\nTask 1 — has_prefix:")
print("Prefix 'app':", trie.has_prefix("app"))       # True
print("Prefix 'bat':", trie.has_prefix("bat"))       # False
print("Prefix 'ban':", trie.has_prefix("ban"))       # True
print("Prefix 'ca':", trie.has_prefix("ca"))         # True

Task 1 — count_words_with_suffix:
Suffix 'e': 1
Suffix 'ion': 1
Suffix 'a': 1
Suffix 'at': 1

Task 1 — has_prefix:
Prefix 'app': True
Prefix 'bat': False
Prefix 'ban': True
Prefix 'ca': True


⛳ Метод count_words_with_suffix повертає кількість слів, що закінчуються на заданий pattern. За відсутності слів повертає 0. Враховує регістр символів. :

✅ Метод has_prefix повертає True, якщо існує хоча б одне слово із заданим префіксом. Повертає False, якщо таких слів немає. Враховує регістр символів.

🦾 Код проходить усі тести.

❕ Обробляються некоректні вхідні дані.

👍 Методи працюють ефективно на великих наборах даних.

---
## Задача 2. Пошук найдовшого спільного префікса

#### Створіть клас `LongestCommonWord`, який наслідує клас Trie, та реалізуйте метод find_longest_common_word, який знаходить найдовший спільний префікс для всіх слів у вхідному масиві рядків strings.

Технічні умови

- Клас LongestCommonWord має успадковувати Trie.
- Вхідний параметр методу find_longest_common_word, strings — масив рядків.
- Метод find_longest_common_word має повертати рядок — найдовший спільний префікс.
- Час виконання — O(S), де S — сумарна довжина всіх рядків.

In [4]:
class LongestCommonWord(Trie):
    def find_longest_common_word(self, strings) -> str:
        if not isinstance(strings, list) or not all(isinstance(s, str) for s in strings):
            raise TypeError("Input must be a list of strings")
        if not strings:
            return ""
        for i, word in enumerate(strings):
            self.put(word, i)

        current = self.root
        prefix = []
        while current and len(current.children) == 1 and current.value is None:
            char = next(iter(current.children))
            prefix.append(char)
            current = current.children[char]
        return "".join(prefix)

### Протестуємо, чи вона працює 🎠

In [8]:
trie = LongestCommonWord()
strings = ["flower", "flow", "flight"]
assert trie.find_longest_common_word(strings) == "fl"
print(strings, "→", trie.find_longest_common_word(strings))       # "fl"

trie = LongestCommonWord()
strings = ["interspecies", "interstellar", "interstate"]
assert trie.find_longest_common_word(strings) == "inters"
print(strings, "→", trie.find_longest_common_word(strings))       # "inters"

trie = LongestCommonWord()
strings = ["dog", "racecar", "car"]
assert trie.find_longest_common_word(strings) == ""
print(strings, "→", trie.find_longest_common_word(strings))       # ""

['flower', 'flow', 'flight'] → fl
['interspecies', 'interstellar', 'interstate'] → inters
['dog', 'racecar', 'car'] → 


Метод find_longest_common_word ✌
  
🪓 повертає найдовший префікс, спільний для всіх слів,

👀 повертає пустий рядок, якщо спільного префікса немає,

❎ коректно обробляє порожній масив або некоректні вхідні дані.

# ✌ Код проходить усі тести 💃